In [1]:
import pandas as pd
import numpy as np
import re as re

In [2]:
df = pd.read_csv("C:/Users/SQL Server Management Studio/Carrier.csv")

In [3]:
# Check datatype of date col
df.iloc[0].date_sub

'5/1/2022'

In [4]:
# Convert it to datetime
df['date'] = pd.to_datetime(df.date_sub)

In [5]:
df.dtypes

date_sub                          object
LMSCarrierConsolidated            object
LMShipClassID                    float64
PO #                             float64
Cost of PO                       float64
IsCancelledFlag                  float64
DirtyShipping_Flag               float64
IsOwnAccountFlag                 float64
date                      datetime64[ns]
dtype: object

In [6]:
# Get the start and end dates for filtering
today = pd.Timestamp('2022-10-28')
period = today.to_period('M')
start_date = (period - 2).to_timestamp()
end_date = period.to_timestamp()

In [7]:
start_date, end_date

(Timestamp('2022-08-01 00:00:00'), Timestamp('2022-10-01 00:00:00'))

In [8]:
# Filter by dates
cond_s = df['date'] > start_date
cond_e = df['date'] <= end_date
cond1 = (
    (df['IsCancelledFlag'] == 0) & 
    (df['DirtyShipping_Flag'] == 0) & 
    (df['IsOwnAccountFlag'] == 0) & 
    (df['LMShipClassID'] != 1)
)
cond2 = (
    (df['LMSCarrierConsolidated'].str.startswith("LP BJ")) | 
    (df['LMSCarrierConsolidated'].str.contains("LP Panth")) | 
    (df['LMSCarrierConsolidated'].str.contains("LP XD")) | 
    (df['LMSCarrierConsolidated'].str.contains("LP Arrow"))
)
df_latest = df[cond_s & cond_e & cond1 & cond2]

In [9]:
# Set datetime index and convert it into period index by month
df_latest = df_latest.set_index('date').to_period('M')

In [10]:
# Bring back index to column for groupby
df3 = df_latest.reset_index()

In [11]:
# Group by month and calculate mean
df3.groupby('date')['Cost of PO'].mean()

date
2022-08    157.455556
2022-09    106.640000
Freq: M, Name: Cost of PO, dtype: float64